# SMS spam classification [sklearn]
Binary classification of sms being spam or not.
Reference: <https://www.kaggle.com/code/faressayah/natural-language-processing-nlp-for-beginners>

## Install necessary packages

In [1]:
!pip install nltk giskard

## Import libraries

In [2]:
import os
from typing import Iterable

import nltk
import string
import sklearn
import pandas as pd
from sklearn import metrics
from nltk.corpus import stopwords
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import FunctionTransformer, LabelEncoder
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer

import giskard
from giskard import wrap_dataset, wrap_model

## Download NLTK corpus

In [3]:
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/mykytaalekseiev/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

## Notebook-level settings

In [4]:
# Necessary for custom preprocessing function transformers.
sklearn.set_config(transform_output="pandas")

## Define constants

In [5]:
# Constants.
TARGET_NAME = "label"
TEXT_COLUMN_NAME = "message"

# Paths.
DATA_DIRECTORY = os.path.join(".", "datasets", "sms_spam_classification_dataset", "spam.csv")

## Load and initially preprocess data

In [6]:
def load_data() -> pd.DataFrame:
    """Load data."""
    df = pd.read_csv(DATA_DIRECTORY, encoding='latin-1')
    df.dropna(how="any", inplace=True, axis=1)
    df.columns = [TARGET_NAME, TEXT_COLUMN_NAME]
    return df

messages_df = load_data()
messages_df.head()

,label,message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [7]:
def preprocess_label(df: pd.DataFrame) -> pd.DataFrame:
    """Change string labels to the integer encoding."""
    df.label = LabelEncoder().fit_transform(df.label)
    return df

messages_df = preprocess_label(messages_df)
messages_df.head()

,label,message
0,0,"Go until jurong point, crazy.. Available only ..."
1,0,Ok lar... Joking wif u oni...
2,1,Free entry in 2 a wkly comp to win FA Cup fina...
3,0,U dun say so early hor... U c already then say...
4,0,"Nah I don't think he goes to usf, he lives aro..."


## Train-test split

In [8]:
X_train, X_test, y_train, y_test = train_test_split(messages_df[[TEXT_COLUMN_NAME]], messages_df[TARGET_NAME], random_state=1)

## Define preprocessing pipeline

In [9]:
def remove_punctuation(df: pd.DataFrame) -> pd.DataFrame:
    """Remove punctuation from text."""
    df[TEXT_COLUMN_NAME] = df[TEXT_COLUMN_NAME].apply(lambda text: text.translate(str.maketrans('', '', string.punctuation)))
    return df

remove_punctuation_transformer = FunctionTransformer(remove_punctuation)

In [10]:
def remove_stop_words(df: pd.DataFrame) -> pd.DataFrame:
    """Remove stopwords from text."""
    _STOPWORDS = stopwords.words('english') + ['u', 'ü', 'ur', '4', '2', 'im', 'dont', 'doin', 'ure']

    df[TEXT_COLUMN_NAME] = df[TEXT_COLUMN_NAME].apply(lambda text: ' '.join([word for word in text.split() if word.lower() not in _STOPWORDS]))
    return df

remove_stop_words_transformer = FunctionTransformer(remove_stop_words)

In [11]:
def adapt_vectorizer_input(df: pd.DataFrame) -> Iterable:
    """Adapt input for the vectorizers.

    The problem is that vectorizers accept iterable, not DataFrame, but Series. Thus, we need to ravel dataframe with text have input single dimension.
    Issue reference: https://stackoverflow.com/questions/50665240/valueerror-found-input-variables-with-inconsistent-numbers-of-samples-1-3185"""

    df = df.iloc[:, 0]
    return df

adapt_vectorizer_input_transformer = FunctionTransformer(adapt_vectorizer_input)

## Define Naive Bayes model pipeline

In [12]:
# Define data preprocessor pipeline.
preprocessor = Pipeline(steps=[
    ("punctuation_remover", remove_punctuation_transformer),
    ("stop_words_remover", remove_stop_words_transformer),
    ("text_vectorizer_adapter", adapt_vectorizer_input_transformer),
    ('bow', CountVectorizer()),
    ('tfid', TfidfTransformer()),
])

# Define general pipeline with data preprocessing and model.
pipeline_naive_bayes = Pipeline(steps=[
    ('preprocessing', preprocessor),
    ('model', MultinomialNB())
])

# Fit model.
pipeline_naive_bayes.fit(X_train, y_train)
y_pred_prob = pipeline_naive_bayes.predict_proba(X_test)[:, 1]

# Get test metric.
metric = metrics.roc_auc_score(y_test, y_pred_prob)
print(f"Test ROC-AUC score: {metric}")

Test ROC-AUC score: 0.9800494641384995


## Define Logistic Regression model pipeline

In [13]:
# Define general pipeline with data preprocessing and model.
pipeline_logistic_regression = Pipeline(steps=[
    ('preprocessing', preprocessor),
    ("logistic_regression", LogisticRegression())
])

# Fit model.
pipeline_logistic_regression.fit(X_train, y_train)
y_pred_prob = pipeline_logistic_regression.predict_proba(X_test)[:, 1]

# Get test metric.
metric = metrics.roc_auc_score(y_test, y_pred_prob)
print(f"Test ROC-AUC score: {metric}")

ROC-AUC score: 0.9842951360263809


## Wrap data and models

In [14]:
# Wrap test dataset.
raw_dataset = pd.concat([X_test, y_test], axis=1)
wrapped_dataset = wrap_dataset(raw_dataset,
                               name="sms_spam",
                               target="label",
                               column_types={"message": "text"})

Your 'pandas.DataFrame' dataset is successfully wrapped by Giskard's 'Dataset' wrapper class.


In [15]:
# Wrap Naive-Bayes model.
wrapped_model_naive_bayes = wrap_model(pipeline_naive_bayes,
                                       model_type="classification",
                                       name="spam_classifier_naive_bayes",
                                       feature_names=X_test.columns,
                                       classification_threshold=0.5)
wrapped_model_naive_bayes.predict(wrapped_dataset)

Your model is successfully wrapped by Giskard's '<class 'abc.ABCMeta'>' wrapper class.


ModelPredictionResults(raw=array([[0.9340599 , 0.0659401 ],
       [0.97673187, 0.02326813],
       [0.96348997, 0.03651003],
       ...,
       [0.9362312 , 0.0637688 ],
       [0.99193208, 0.00806792],
       [0.9862079 , 0.0137921 ]]), prediction=array([0, 0, 0, ..., 0, 0, 0]), raw_prediction=array([0, 0, 0, ..., 0, 0, 0]), probabilities=array([0.9340599 , 0.97673187, 0.96348997, ..., 0.9362312 , 0.99193208,
       0.9862079 ]), all_predictions=             0         1
0     0.934060  0.065940
1     0.976732  0.023268
2     0.963490  0.036510
3     0.957491  0.042509
4     0.928919  0.071081
...        ...       ...
1388  0.996038  0.003962
1389  0.998073  0.001927
1390  0.936231  0.063769
1391  0.991932  0.008068
1392  0.986208  0.013792

[1393 rows x 2 columns])

In [16]:
# Wrap Logistic Regression model.
wrapped_model_logistic_regression = wrap_model(pipeline_logistic_regression,
                                               model_type="classification",
                                               name="spam_classifier_logistic_regression",
                                               feature_names=X_test.columns,
                                               classification_threshold=0.5)
wrapped_model_logistic_regression.predict(wrapped_dataset)

Your model is successfully wrapped by Giskard's '<class 'abc.ABCMeta'>' wrapper class.


ModelPredictionResults(raw=array([[0.926392  , 0.073608  ],
       [0.94690471, 0.05309529],
       [0.84887983, 0.15112017],
       ...,
       [0.91830997, 0.08169003],
       [0.95666017, 0.04333983],
       [0.94902435, 0.05097565]]), prediction=array([0, 0, 0, ..., 0, 0, 0]), raw_prediction=array([0, 0, 0, ..., 0, 0, 0]), probabilities=array([0.926392  , 0.94690471, 0.84887983, ..., 0.91830997, 0.95666017,
       0.94902435]), all_predictions=             0         1
0     0.926392  0.073608
1     0.946905  0.053095
2     0.848880  0.151120
3     0.947935  0.052065
4     0.637550  0.362450
...        ...       ...
1388  0.960758  0.039242
1389  0.988461  0.011539
1390  0.918310  0.081690
1391  0.956660  0.043340
1392  0.949024  0.050976

[1393 rows x 2 columns])

## Scan models

In [17]:
# Scan Naive Bayes model.
naive_bayes_scan = giskard.scan(wrapped_model_naive_bayes, wrapped_dataset)

/Users/mykytaalekseiev/IdeaProjects/GiskardExamples/venv/lib/python3.10/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


Hint: "Your target variable values are numeric. It is recommended to have Human readable string as your target values to make results more understandable in Giskard."
Your model is successfully validated.


In [18]:
display(naive_bayes_scan)

<iframe srcdoc="
<script src="https://cdn.tailwindcss.com"></script>
<script>
 tailwind.config = {
 darkMode: 'class'
 }
</script>
<style>
 table.dataframe {
 width: 100%;
 max-width: 100%;
 overflow: auto;
 }

 .dataframe tr {
 border-bottom: 1px solid #555;
 }

 .dataframe td,
 .dataframe th {
 padding: 0.5rem;
 }
</style>



<div class="dark">
 <div id="gsk-scan" class="dark:text-white dark:bg-zinc-900">
 <!-- TAB HEADER -->
<div class="flex items-center items-stretch h-11">
 <div class="flex items-center px-4 dark:fill-white border-b border-gray-500 h-11">
 <svg xmlns="http://www.w3.org/2000/svg" xml:space="preserve" viewBox="0 0 410 213" height="15">
 <path
 d="M318.78 21.94a59.439 59.439 0 0 1 36.45-12.56v.03c7.09-.1 14.13 1.2 20.71 3.84 6.58 2.64 12.57 6.56 17.62 11.54s9.06 10.91 11.8 17.45a53.44 53.44 0 0 1 0 41.3 53.588 53.588 0 0 1-11.8 17.45c-5.05 4.98-11.04 8.9-17.62 11.54s-13.62 3.95-20.71 3.84l-67.92.16-35.87 49.11-135.24 46.84 40.81-47.24a139.229 139.229 0 0 1-66.06-23.8H45.11L0 101.46h55.73C60.38 44.84 115.97 0 183.59 0c31.71 0 63.58 9.86 87.47 27.04a105.149 105.149 0 0 1 26.65 27.18 59.4 59.4 0 0 1 21.07-32.28zM99.85 46.03l55.4 55.45h21.31l-64.81-64.87c-4.18 2.86-8.15 6.01-11.9 9.42zm57.78-28.45 83.9 83.9h21.29l-86.21-86.22c-6.37.32-12.71 1.1-18.98 2.32zm-92.84 98.94H39.72l11.1 9.82h22.44c-3-3.12-5.83-6.4-8.47-9.82zm134.27 0H84.8l-.03.01c20.8 20.91 51.79 33.67 84.66 34.3l29.63-34.31zm-34.81 63.35 77.79-26.94 26.6-36.39h-49.68l-54.71 63.33zm146.77-78.65h46.2l.05.01c9.98-.55 19.36-4.97 26.14-12.32a38.409 38.409 0 0 0 10.16-27.05 38.44 38.44 0 0 0-11.56-26.48 38.405 38.405 0 0 0-26.74-10.94A44.287 44.287 0 0 0 324 37.41a44.268 44.268 0 0 0-12.97 31.28v32.53zm51.82-52.83c5.27.52 9.12 5.22 8.6 10.49-.52 5.27-5.22 9.12-10.49 8.6s-9.12-5.22-8.6-10.49c.53-5.27 5.22-9.12 10.49-8.6z"
 style="fill-rule:evenodd;clip-rule:evenodd" />
 </svg>
 <div class="ml-4 py-2">
 <span class="uppercase text-sm">8 issues detected</span>
 </div>
 </div>

 
 <div data-tab-target="Model_bias" class="h-12 group active">
 <div
 class="group-[.active]:bg-zinc-800 group-[.active]:border-b-zinc-800 group-[.active]:border-gray-500 border-l px-3 py-2 border-r border-t border-b border-gray-600 border-b-gray-500 h-11">
 Model bias
 
 
 <span class="ml-1 rounded-full text-xs min-w-4 min-h-4 px-1 py-0.5 inline-block text-center bg-red-400">
 
 8
 </span>
 
 </div>
 </div>
 

 <div class="flex-grow border-b border-gray-500 h-11"></div>
</div>
<!-- TAB HEADER END -->
 
 <div id="Model_bias" role="tabpanel" class="dark:text-white dark:bg-zinc-800 p-4 pt-4 mb-4">
 
 <div class="p-3 mt-2 bg-amber-100/40 rounded-sm w-full flex align-middle">
 <div class="text-amber-100">
 <svg xmlns="http://www.w3.org/2000/svg" fill="none" viewBox="0 0 24 24" stroke-width="1.5"
 stroke="currentColor" class="mt-0.5 w-6 h-6">
 <path stroke-linecap="round" stroke-linejoin="round"
 d="M12 9v3.75m-9.303 3.376c-.866 1.5.217 3.374 1.948 3.374h14.71c1.73 0 2.813-1.874 1.948-3.374L13.949 3.378c-.866-1.5-3.032-1.5-3.898 0L2.697 16.126zM12 15.75h.007v.008H12v-.008z" />
 </svg>
 </div>
 <p class="ml-2 my-1 text-amber-100 text-sm">
 We detected some spots in your data where the model has a tendency to make incorrect
 predictions.
 </p>
 </div>
 

 
 <div class="flex items-center space-x-1">
 <h2 class="uppercase my-4 mr-2 font-medium">Issues</h2>
 
 <span class="text-xs border rounded px-1 uppercase text-red-400 border-red-400">7 major</span>
 
 
 <span class="text-xs border rounded px-1 uppercase text-amber-200 border-amber-200">1 medium</span>
 
 </div>

 <!-- ISSUES TABLE -->
<div class="mb-4">
 <table class="table-auto w-full text-white">
 
 <tbody class="first:border-t border-b border-zinc-400">
 <tr class="gsk-issue text-sm group peer text-left cursor-pointer hover:bg-zinc-700">
 <td class="p-3">
 <svg xmlns="http://www.w3.org/2000/svg" fill="none" viewBox="0 0 24 24" stroke-width="1.5"
 stroke="currentColor" class="w-4 h-4 group-[.open]:rotate-90">
 <path stroke-linecap="round" 

In [19]:
# Scan Logistic regression model.
logistic_regression_scan = giskard.scan(wrapped_model_logistic_regression, wrapped_dataset)

Hint: "Your target variable values are numeric. It is recommended to have Human readable string as your target values to make results more understandable in Giskard."
Your model is successfully validated.


In [20]:
display(logistic_regression_scan)

<iframe srcdoc="
<script src="https://cdn.tailwindcss.com"></script>
<script>
 tailwind.config = {
 darkMode: 'class'
 }
</script>
<style>
 table.dataframe {
 width: 100%;
 max-width: 100%;
 overflow: auto;
 }

 .dataframe tr {
 border-bottom: 1px solid #555;
 }

 .dataframe td,
 .dataframe th {
 padding: 0.5rem;
 }
</style>



<div class="dark">
 <div id="gsk-scan" class="dark:text-white dark:bg-zinc-900">
 <!-- TAB HEADER -->
<div class="flex items-center items-stretch h-11">
 <div class="flex items-center px-4 dark:fill-white border-b border-gray-500 h-11">
 <svg xmlns="http://www.w3.org/2000/svg" xml:space="preserve" viewBox="0 0 410 213" height="15">
 <path
 d="M318.78 21.94a59.439 59.439 0 0 1 36.45-12.56v.03c7.09-.1 14.13 1.2 20.71 3.84 6.58 2.64 12.57 6.56 17.62 11.54s9.06 10.91 11.8 17.45a53.44 53.44 0 0 1 0 41.3 53.588 53.588 0 0 1-11.8 17.45c-5.05 4.98-11.04 8.9-17.62 11.54s-13.62 3.95-20.71 3.84l-67.92.16-35.87 49.11-135.24 46.84 40.81-47.24a139.229 139.229 0 0 1-66.06-23.8H45.11L0 101.46h55.73C60.38 44.84 115.97 0 183.59 0c31.71 0 63.58 9.86 87.47 27.04a105.149 105.149 0 0 1 26.65 27.18 59.4 59.4 0 0 1 21.07-32.28zM99.85 46.03l55.4 55.45h21.31l-64.81-64.87c-4.18 2.86-8.15 6.01-11.9 9.42zm57.78-28.45 83.9 83.9h21.29l-86.21-86.22c-6.37.32-12.71 1.1-18.98 2.32zm-92.84 98.94H39.72l11.1 9.82h22.44c-3-3.12-5.83-6.4-8.47-9.82zm134.27 0H84.8l-.03.01c20.8 20.91 51.79 33.67 84.66 34.3l29.63-34.31zm-34.81 63.35 77.79-26.94 26.6-36.39h-49.68l-54.71 63.33zm146.77-78.65h46.2l.05.01c9.98-.55 19.36-4.97 26.14-12.32a38.409 38.409 0 0 0 10.16-27.05 38.44 38.44 0 0 0-11.56-26.48 38.405 38.405 0 0 0-26.74-10.94A44.287 44.287 0 0 0 324 37.41a44.268 44.268 0 0 0-12.97 31.28v32.53zm51.82-52.83c5.27.52 9.12 5.22 8.6 10.49-.52 5.27-5.22 9.12-10.49 8.6s-9.12-5.22-8.6-10.49c.53-5.27 5.22-9.12 10.49-8.6z"
 style="fill-rule:evenodd;clip-rule:evenodd" />
 </svg>
 <div class="ml-4 py-2">
 <span class="uppercase text-sm">8 issues detected</span>
 </div>
 </div>

 
 <div data-tab-target="Model_bias" class="h-12 group active">
 <div
 class="group-[.active]:bg-zinc-800 group-[.active]:border-b-zinc-800 group-[.active]:border-gray-500 border-l px-3 py-2 border-r border-t border-b border-gray-600 border-b-gray-500 h-11">
 Model bias
 
 
 <span class="ml-1 rounded-full text-xs min-w-4 min-h-4 px-1 py-0.5 inline-block text-center bg-red-400">
 
 8
 </span>
 
 </div>
 </div>
 

 <div class="flex-grow border-b border-gray-500 h-11"></div>
</div>
<!-- TAB HEADER END -->
 
 <div id="Model_bias" role="tabpanel" class="dark:text-white dark:bg-zinc-800 p-4 pt-4 mb-4">
 
 <div class="p-3 mt-2 bg-amber-100/40 rounded-sm w-full flex align-middle">
 <div class="text-amber-100">
 <svg xmlns="http://www.w3.org/2000/svg" fill="none" viewBox="0 0 24 24" stroke-width="1.5"
 stroke="currentColor" class="mt-0.5 w-6 h-6">
 <path stroke-linecap="round" stroke-linejoin="round"
 d="M12 9v3.75m-9.303 3.376c-.866 1.5.217 3.374 1.948 3.374h14.71c1.73 0 2.813-1.874 1.948-3.374L13.949 3.378c-.866-1.5-3.032-1.5-3.898 0L2.697 16.126zM12 15.75h.007v.008H12v-.008z" />
 </svg>
 </div>
 <p class="ml-2 my-1 text-amber-100 text-sm">
 We detected some spots in your data where the model has a tendency to make incorrect
 predictions.
 </p>
 </div>
 

 
 <div class="flex items-center space-x-1">
 <h2 class="uppercase my-4 mr-2 font-medium">Issues</h2>
 
 <span class="text-xs border rounded px-1 uppercase text-red-400 border-red-400">8 major</span>
 
 
 </div>

 <!-- ISSUES TABLE -->
<div class="mb-4">
 <table class="table-auto w-full text-white">
 
 <tbody class="first:border-t border-b border-zinc-400">
 <tr class="gsk-issue text-sm group peer text-left cursor-pointer hover:bg-zinc-700">
 <td class="p-3">
 <svg xmlns="http://www.w3.org/2000/svg" fill="none" viewBox="0 0 24 24" stroke-width="1.5"
 stroke="currentColor" class="w-4 h-4 group-[.open]:rotate-90">
 <path stroke-linecap="round" stroke-linejoin="round" d="M8.25 4.5l7.5 7.5-7.5 7.5" />
 </svg>
 </td>
 <td class="p-3">
 <code class=